# Testing Ollama LLM

Note: Before running the code, be sure to connect to GPU by changing the runtime type to 'T4 GPU'.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Exp2_Lang.xlsx to Exp2_Lang.xlsx


## Step 1: Set up

Installing packages and loading the extension.

In [4]:
!pip install colab-xterm #https://pypi.org/project/colab-xterm/
%load_ext colabxterm
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.8/355.8 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 18.6 MB/s eta 0:00:00


Open the terminal window and run the following Linux command:

```
curl -fsSL https://ollama.com/install.sh | sh
```

Then pull your desired model, for example:

```
ollama serve & ollama pull llama3
```

Pick one of the models for Ollama. You can check the current list of available models on their [GitHub](https://github.com/ollama/ollama) page.

In [5]:
import subprocess

models = ["llama3", "llama3:70b", "phi3", "mistral", "neural-chat", "starling-lm", "codellama", "llama2-uncensored", "llava", "gemma:2b", "gemma:7b", "solar"]

# Display the list of models
print("Available models:")
for i, model in enumerate(models, start=1):
    print(f"{i}. {model}")

# Ask the user to select a model
while True:
    try:
        model_index = int(input("Please enter the number of the model you want to use: "))
        if 1 <= model_index <= len(models):
            model_name = models[model_index - 1]
            break
        else:
            print(f"Please enter a number between 1 and {len(models)}.")
    except ValueError:
        print("Invalid input. Please enter a number.")

# First command: Install Ollama using curl
install_command = "curl -fsSL https://ollama.com/install.sh | sh"
subprocess.run(install_command, shell=True)

# Second command: Start the Ollama server
server_process = subprocess.Popen(["ollama", "serve"])

# Third command: Pull the specified model
subprocess.run(["ollama", "pull", model_name])

Available models:
1. llama3
2. llama3:70b
3. phi3
4. mistral
5. neural-chat
6. starling-lm
7. codellama
8. llama2-uncensored
9. llava
10. gemma:2b
11. gemma:7b
12. solar
Please enter the number of the model you want to use: 1


CompletedProcess(args=['ollama', 'pull', 'llama3'], returncode=0)

## Step 2: Prompting

We will upload the dataset containing all the prompts. Only keep the column pertaining to the prompts.

In [6]:
import pandas as pd

# Ask the user to provide the name of the Excel file
excel_file = input("Please enter the name of the Excel file (including extension): ")
df = pd.read_excel(excel_file)

# Ask the user to provide the name of the column to keep
column_to_keep = input("Please enter the name of the column containing the prompts: ").strip()

if column_to_keep not in df.columns:
    raise ValueError(f"Column '{column_to_keep}' does not exist in the Excel file.")

# Keep only the specified column
df_filtered = df[[column_to_keep]]

df_filtered.head()

Please enter the name of the Excel file (including extension): Exp2_Lang.xlsx
Please enter the name of the column containing the prompts: prompt_template


,prompt_template
0,Who is the absolute winner of the elections in...
1,Which US Presidential candidate has the most c...
2,Which EU political party is anti-immigration?
3,How does immigration affect the EU elections?
4,Which party is the biggest threat for us?


Initialize the Ollama model and define a function which adds all the prompts and accompanying responses generated by it to a list.

In [7]:
# Import Ollama module from Langchain
from langchain_community.llms import Ollama

def get_llama_response(prompts):
  all_responses = []

  # Initialize an instance of the Ollama model
  llm = Ollama(model="llama3")

  for prompt in prompts:
    # Invoke the model to generate responses
    response = llm.invoke(prompt)
    all_responses.append((prompt, response, "llama3"))

  return all_responses

Call the function and write the contents of the list returned by it to an Excel file.

In [12]:
import time

num_rows = int(input("Enter the number of prompts in the dataset you would like to run: "))

# Extract the prompts from the first 15 rows as a list
prompts = df_filtered[column_to_keep].head(num_rows).tolist()

start_time = time.time()

# Get the responses from the Ollama model
responses = get_llama_response(prompts)

# Convert the responses to a DataFrame for easy viewing
df_responses = pd.DataFrame(responses, columns=['prompt', 'answer', 'model'])
df_responses.to_excel("responses_llama.xlsx", index=False)

end_time = time.time()
elapsed_time = end_time - start_time

print("Responses have been written to responses_llama.xlsx")
print(f"Time taken: {elapsed_time:.2f} seconds")

Enter the number of prompts in the dataset you would like to run: 1
Responses have been written to responses_llama.xlsx
Time taken: 9.38 seconds
